# # Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

# Introduction

New York City is a collection of many neighbourhoods scattered among the city’s five boroughs—Manhattan, Brooklyn, the Bronx, Queens, and Staten Island—each exhibiting its own lifestyle. It is the most populous and the most international city in the country. Furthermore, New York is the most ethnically diverse, religiously varied, commercially driven, famously congested, and, in the eyes of many, the most attractive urban centre in the country.

Hence, opening any new business should be carefully studied and a strategic plan should be developped in order to guarantee the long term succes. Indeed, this requires a careful understanding of the diffrent neighbourhoods of the city, the ethnicity of the people and most important their habits and tastes.


# Business problem

We can say that healthcare is one of the most valuable industries in the World. This is due to the spread of the fast food culture which leads to many diseases and health problems. Hence, many people are trying to find methods in order to avoid falling into these problems. We can say that the preventive cure exists in two major categories: gym and healthy food.

In terms of healthy food, vegetrian restaurants come in the top of the list. Starting a vegterian restaurant can be a great business opportunity, but we need to distinguish ourselves from others to enjoy long-term success. We think that by exploring the neighbourhoods and their different venues and by emphesizing on the locations where gym centers are popular will lead us to the right place of our presumed vegetarian restaurants. This is because people who frequently visit gym are likely interested in eating healthy food.

We will content ourselves to provide this study for the city of New York and in particular to Manhattan neighbourhood since our client is interested in this area.


# Data

We will rely on the publicly available data of New York city available online: https://geo.nyu.edu/catalog/nyu_2451_34572

This dataset contains 5 boroughs and 306 neighborhoods. Now since our methodology is to search for gym/fitness center venues and try to plug our vegeterian restaurant nearby, we will use this website:https://developer.foursquare.com/docs/resources/categories in order to find gym/fitness center category id. We found that this id = 4bf58dd8d48988d175941735


# Methodology

Our methodology consists of loading and cleaning the dataset, then performing k-means statisical analysis in two stages. First, we consider venues with category = gym/fitness. Second, we do clustering regardless of the category type. Then we decide if a neighborhood needs a vegetarian restaurant, by inspecting if it appears in a cluster with high concentation in gym/fitness venues (i.e. by looking for clustering in Stage 1) and low concentration in restaurants, especially vegetarian restaurants (i.e. by looking for clustering in Stage 2). 


# Code

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Libraries imported.


In [5]:
# download dataset and open it
with open('nyu_2451_34572-geojson.json') as json_data:
    newyork_data = json.load(json_data)

In [7]:
#constructing the dataframe
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [8]:
# get tha Lat/long of New York city
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [9]:
# Filtering by Manhattan Borough
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [11]:
# Getting map of New york
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers (from Manhattan) to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [12]:
# Now defining a function to use Foursquare API
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [13]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define raduis
CLIENT_ID = 'KGGGPBGME2GPYNSNBFG0RJWJJG3X0FTBQ3U0BDRUSV0BQSOI'
CLIENT_SECRET = 'FRPR5WN43GBGX4SVVLDP1RQ4OERGM5CCPCQ4HFTGGZ220QQJ'
VERSION = '20180605'

In [14]:
# First Get neighborhoods with gym/fitness venues
#https://developer.foursquare.com/docs/resources/categories
#gym = 4bf58dd8d48988d175941735
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_gym = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=500, categoryIds='4bf58dd8d48988d175941735')
newyork_venues_gym.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Bronx Boxing,40.875671,-73.908355,Boxing Gym
1,Marble Hill,40.876551,-73.91066,Planet Fitness,40.874088,-73.909137,Gym / Fitness Center
2,Marble Hill,40.876551,-73.91066,Astral Fitness & Wellness Center,40.876705,-73.906372,Gym
3,Marble Hill,40.876551,-73.91066,Blink Fitness Riverdale,40.877147,-73.905837,Gym
4,Marble Hill,40.876551,-73.91066,Weight Watchers,40.873341,-73.911362,Weight Loss Center


In [15]:
newyork_venues_gym.shape

(1553, 7)

In [35]:
# second get neighborhoods with different types of venues
newyork_venues_all = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=500)
newyork_venues_all.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Marble Hill,40.876111,-73.911111,Neighborhood
1,Marble Hill,40.876551,-73.91066,108 Marblehill,40.876605,-73.909430,Housing Development
2,Marble Hill,40.876551,-73.91066,MTA Subway - 225th St/Marble Hill (1),40.874486,-73.909589,Metro Station
3,Marble Hill,40.876551,-73.91066,Wine & Liquors,40.874535,-73.909832,Wine Shop
4,Marble Hill,40.876551,-73.91066,St. Johns Roman Catholic Church,40.876174,-73.909795,Church


In [16]:
#defining a function to add gyms to maps
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [18]:
map_newyork_gym = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_gym, 'red', map_newyork_gym)

map_newyork_gym

In [19]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [20]:
manhattan_grouped = newyork_venues_gym.groupby('Neighborhood').count()
manhattan_grouped

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,50,50,50,50,50,50
Carnegie Hill,49,49,49,49,49,49
Central Harlem,13,13,13,13,13,13
Chelsea,48,48,48,48,48,48
Chinatown,45,45,45,45,45,45
Civic Center,50,50,50,50,50,50
Clinton,50,50,50,50,50,50
East Harlem,16,16,16,16,16,16
East Village,48,48,48,48,48,48


In [36]:
# Now doing the same for neighborhoods with venues of any kind
manhattan_all_grouped = newyork_venues_all.groupby('Neighborhood').count()
manhattan_all_grouped

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,96,96,96,96,96,96
Carnegie Hill,94,94,94,94,94,94
Central Harlem,93,93,93,93,93,93
Chelsea,85,85,85,85,85,85
Chinatown,95,95,95,95,95,95
Civic Center,91,91,91,91,91,91
Clinton,92,92,92,92,92,92
East Harlem,83,83,83,83,83,83
East Village,81,81,81,81,81,81


In [21]:
# Analyzing each neigbourhood 
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_gym[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_gym['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Acupuncturist,Athletics & Sports,Basketball Court,Boxing Gym,Bridge,Building,Climbing Gym,Club House,College Gym,Community Center,Corporate Amenity,Cycle Studio,Dance Studio,Doctor's Office,General College & University,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hotel,Martial Arts Dojo,Medical Center,Non-Profit,Office,Outdoor Gym,Physical Therapist,Pilates Studio,Pool,Recreation Center,Residential Building (Apartment / Condo),Spa,Spiritual Center,Tennis Court,Tennis Stadium,Track,Weight Loss Center,Women's Store,Yoga Studio
0,Marble Hill,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [37]:
# Same for naighborhoods with venues of any category
# Analyzing each neigbourhood
# one hot encoding
manhattan_onehot_all = pd.get_dummies(newyork_venues_all[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot_all['Neighborhood'] = newyork_venues_all['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns_all = [manhattan_onehot_all.columns[-1]] + list(manhattan_onehot_all.columns[:-1])
manhattan_onehot_all = manhattan_onehot_all[fixed_columns_all]

manhattan_onehot_all.head()

,Yoga Studio,Accessories Store,Acupuncturist,Advertising Agency,Afghan Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Art Studio,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bike Shop,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Campaign Office,Candy Store,Caribbean Restaurant,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Church,City,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College & University,College Academic Building,College Administrative Building,College Arts Building,College Auditorium,College Bookstore,College Cafeteria,College Classroom,College Communications Building,College Gym,College Lab,College Library,College Quad,College Rec Center,College Residence Hall,College Science Building,College Theater,Comedy Club,Community Center,Community College,Concert Hall,Conference Room,Convenience Store,Convention Center,Cooking School,Corporate Cafeteria,Cosmetics Shop,County,Coworking Space,Credit Union,Creperie,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Driving School,Dry Cleaner,Dumpling Restaurant,EV Charging Station,Electronics Store,Elementary School,Embassy / Consulate,Empanada Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fire Station,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Funeral Home,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,High School,Himalayan Restaurant,Historic Site,History Museum,Hookah Bar,Hospital,Hospital Ward,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Laboratory,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical Lab,Medical School,Mediterranean Restaurant,Meeting Room,Men's Store,Mental Health Office,Metro Station,Mexican Restaurant,Middle School,Military Base,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Mosque,Motel,Mountain,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,Nursery School,Office,Opera House,Optical Shop,Other Event,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,Physical Therapist,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Police Station,Poo

In [22]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Acupuncturist,Athletics & Sports,Basketball Court,Boxing Gym,Bridge,Building,Climbing Gym,Club House,College Gym,Community Center,Corporate Amenity,Cycle Studio,Dance Studio,Doctor's Office,General College & University,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hotel,Martial Arts Dojo,Medical Center,Non-Profit,Office,Outdoor Gym,Physical Therapist,Pilates Studio,Pool,Recreation Center,Residential Building (Apartment / Condo),Spa,Spiritual Center,Tennis Court,Tennis Stadium,Track,Weight Loss Center,Women's Store,Yoga Studio
0,Battery Park City,0.00,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.02,0.020000,0.00,0.000000,0.000000,0.700000,0.160000,0.020000,0.020000,0.00,0.000000,0.000000,0.00,0.020000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.020000
1,Carnegie Hill,0.00,0.000000,0.000000,0.000000,0.000000,0.020408,0.020408,0.00,0.00,0.020408,0.00,0.000000,0.00,0.000000,0.000000,0.367347,0.306122,0.020408,0.000000,0.00,0.081633,0.000000,0.00,0.000000,0.000000,0.00,0.040816,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.020408,0.020408,0.000000,0.081633
2,Central Harlem,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.076923,0.00,0.000000,0.076923,0.461538,0.307692,0.000000,0.000000,0.00,0.076923,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000
3,Chelsea,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.00,0.00,0.000000,0.00,0.041667,0.00,0.000000,0.000000,0.166667,0.416667,0.000000,0.020833,0.00,0.083333,0.000000,0.00,0.000000,0.000000,0.00,0.020833,0.000000,0.020833,0.000000,0.020833,0.00,0.00,0.0,0.020833,0.000000,0.000000,0.166667
4,Chinatown,0.00,0.022222,0.000000,0.022222,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.288889,0.311111,0.000000,0.000000,0.00,0.200000,0.000000,0.00,0.000000,0.000000,0.00,0.066667,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.088889
5,Civic Center,0.02,0.040000,0.000000,0.080000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.200000,0.340000,0.020000,0.000000,0.00,0.080000,0.020000,0.00,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.020000,0.160000
6,Clinton,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.600000,0.280000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.02,0.0,0.000000,0.000000,0.000000,0.060000
7,East Harlem,0.00,0.000000,0.000000,0.062500,0.000000,0.062500,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.312500,0.250000,0.000000,0.000000,0.00,0.187500,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.062500,0.000000,0.000000,0.062500
8,East Village,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.395833,0.229167,0.000000,0.020833,0.00,0.041667,0.000000,0.00,0.000000,0.041667,0.00,0.083333,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.020833,0.000000,0.000000,0.166667
9,Financial District,0.00,0.000000,0.000000,0.060000,0.000000,0.000000,0.000000,0.00,0.02,0.000000,0.00,0.020000,0.00,0.020000,0.000000,0.500000,0.300000,0.000000,0.000000,0.00,0.000000,0.020000,0.00,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.020000,0.000000,0.020000


In [38]:
# for neighborhoods with venues of any category
manhattan_all_grouped = manhattan_onehot_all.groupby('Neighborhood').mean().reset_index()
manhattan_all_grouped

,Neighborhood,Yoga Studio,Accessories Store,Acupuncturist,Advertising Agency,Afghan Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Art Studio,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bike Shop,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Campaign Office,Candy Store,Caribbean Restaurant,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Church,City,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College & University,College Academic Building,College Administrative Building,College Arts Building,College Auditorium,College Bookstore,College Cafeteria,College Classroom,College Communications Building,College Gym,College Lab,College Library,College Quad,College Rec Center,College Residence Hall,College Science Building,College Theater,Comedy Club,Community Center,Community College,Concert Hall,Conference Room,Convenience Store,Convention Center,Cooking School,Corporate Cafeteria,Cosmetics Shop,County,Coworking Space,Credit Union,Creperie,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Driving School,Dry Cleaner,Dumpling Restaurant,EV Charging Station,Electronics Store,Elementary School,Embassy / Consulate,Empanada Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fire Station,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Funeral Home,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,High School,Himalayan Restaurant,Historic Site,History Museum,Hookah Bar,Hospital,Hospital Ward,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Laboratory,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical Lab,Medical School,Mediterranean Restaurant,Meeting Room,Men's Store,Mental Health Office,Metro Station,Mexican Restaurant,Middle School,Military Base,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Mosque,Motel,Mountain,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,Nursery School,Office,Opera House,Optical Shop,Other Event,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,Physical Therapist,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Police Station,Poo

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Gym,Gym / Fitness Center,Yoga Studio,Office,Boxing Gym,Corporate Amenity,Cycle Studio,Gym Pool,Gymnastics Gym,Recreation Center
1,Carnegie Hill,Gym,Gym / Fitness Center,Martial Arts Dojo,Yoga Studio,Pilates Studio,Climbing Gym,Gym Pool,Weight Loss Center,Track,Community Center
2,Central Harlem,Gym,Gym / Fitness Center,Martial Arts Dojo,General College & University,Cycle Studio,Community Center,Doctor's Office,Dance Studio,Corporate Amenity,Yoga Studio
3,Chelsea,Gym / Fitness Center,Yoga Studio,Gym,Martial Arts Dojo,Cycle Studio,Recreation Center,Climbing Gym,Pilates Studio,Gymnastics Gym,Spa
4,Chinatown,Gym / Fitness Center,Gym,Martial Arts Dojo,Yoga Studio,Pilates Studio,Boxing Gym,Athletics & Sports,Bridge,Building,Climbing Gym


In [39]:
# Getting 10 top most common venues for neighborhoods with venues of any category
# create a new dataframe
# create columns according to number of top venues
columns_all = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns_all.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns_all.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted_all = pd.DataFrame(columns=columns_all)
neighborhoods_venues_sorted_all['Neighborhood'] = manhattan_all_grouped['Neighborhood']

for ind in np.arange(manhattan_all_grouped.shape[0]):
    neighborhoods_venues_sorted_all.iloc[ind, 1:] = return_most_common_venues(manhattan_all_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_all.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Residential Building (Apartment / Condo),Boat or Ferry,Park,Harbor / Marina,Office,Food Truck,Building,Hot Dog Joint,Bar,Dog Run
1,Carnegie Hill,Residential Building (Apartment / Condo),Salon / Barbershop,Building,Event Space,Coffee Shop,Doctor's Office,Gym / Fitness Center,Nail Salon,Pizza Place,Gym
2,Central Harlem,Salon / Barbershop,Residential Building (Apartment / Condo),Mountain,Gym / Fitness Center,Bus Line,Bus Stop,Non-Profit,Church,Cocktail Bar,Building
3,Chelsea,High School,Building,Residential Building (Apartment / Condo),Deli / Bodega,Office,Art Gallery,Pet Store,Pharmacy,Music Venue,Laundry Service
4,Chinatown,Chinese Restaurant,Miscellaneous Shop,Bus Station,Bakery,Bridge,Noodle House,Park,Arts & Crafts Store,Bus Line,Ice Cream Shop


# Clustering based on venue category = gym/fitness center

In [25]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 2, 2, 4, 1, 4, 3, 1, 1, 2])

In [26]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Yoga Studio,Gym / Fitness Center,Gym,Boxing Gym,Martial Arts Dojo,Weight Loss Center,Tennis Stadium,Athletics & Sports,Corporate Amenity,General College & University
1,Manhattan,Chinatown,40.715618,-73.994279,1,Gym / Fitness Center,Gym,Martial Arts Dojo,Yoga Studio,Pilates Studio,Boxing Gym,Athletics & Sports,Bridge,Building,Climbing Gym
2,Manhattan,Washington Heights,40.851903,-73.936900,3,Gym,Gym / Fitness Center,Pilates Studio,Community Center,General College & University,Doctor's Office,Dance Studio,Cycle Studio,Corporate Amenity,Yoga Studio
3,Manhattan,Inwood,40.867684,-73.921210,1,Gym,Yoga Studio,Pilates Studio,Gym / Fitness Center,Boxing Gym,Bridge,Building,Climbing Gym,Club House,Gym Pool
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Yoga Studio,Gym,Gym / Fitness Center,Basketball Court,Community Center,General College & University,Doctor's Office,Dance Studio,Cycle Studio,Corporate Amenity


In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster 1

In [28]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Hamilton Heights,Yoga Studio,Gym,Gym / Fitness Center,Basketball Court,Community Center,General College & University,Doctor's Office,Dance Studio,Cycle Studio,Corporate Amenity


## Cluster 2

In [30]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Yoga Studio,Gym / Fitness Center,Gym,Boxing Gym,Martial Arts Dojo,Weight Loss Center,Tennis Stadium,Athletics & Sports,Corporate Amenity,General College & University
1,Chinatown,Gym / Fitness Center,Gym,Martial Arts Dojo,Yoga Studio,Pilates Studio,Boxing Gym,Athletics & Sports,Bridge,Building,Climbing Gym
3,Inwood,Gym,Yoga Studio,Pilates Studio,Gym / Fitness Center,Boxing Gym,Bridge,Building,Climbing Gym,Club House,Gym Pool
5,Manhattanville,Gym / Fitness Center,Gym,Yoga Studio,Climbing Gym,Pilates Studio,Basketball Court,Boxing Gym,Bridge,Building,Club House
7,East Harlem,Gym,Gym / Fitness Center,Martial Arts Dojo,Boxing Gym,Building,Yoga Studio,Track,Bridge,Basketball Court,Climbing Gym
18,Greenwich Village,Gym,Gym / Fitness Center,Yoga Studio,Pilates Studio,Cycle Studio,Spa,Martial Arts Dojo,Boxing Gym,Medical Center,Basketball Court
19,East Village,Gym,Gym / Fitness Center,Yoga Studio,Pilates Studio,Martial Arts Dojo,Outdoor Gym,Gymnastics Gym,Track,Spa,General College & University
20,Lower East Side,Gym,Gym / Fitness Center,Martial Arts Dojo,Yoga Studio,Pool,Cycle Studio,Bridge,Track,Pilates Studio,Community Center
22,Little Italy,Yoga Studio,Gym,Gym / Fitness Center,Pilates Studio,Martial Arts Dojo,Boxing Gym,Cycle Studio,Athletics & Sports,Corporate Amenity,General College & University
23,Soho,Gym,Gym / Fitness Center,Yoga Studio,Martial Arts Dojo,Pilates Studio,Cycle Studio,Boxing Gym,Medical Center,Office,Basketball Court


## Cluster 3

In [32]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Central Harlem,Gym,Gym / Fitness Center,Martial Arts Dojo,General College & University,Cycle Studio,Community Center,Doctor's Office,Dance Studio,Corporate Amenity,Yoga Studio
9,Yorkville,Gym,Gym / Fitness Center,Martial Arts Dojo,Pilates Studio,Boxing Gym,Gymnastics Gym,Weight Loss Center,Pool,Bridge,Building
12,Upper West Side,Gym,Gym / Fitness Center,Yoga Studio,Pilates Studio,Martial Arts Dojo,Boxing Gym,Cycle Studio,Gymnastics Gym,Residential Building (Apartment / Condo),Recreation Center
13,Lincoln Square,Gym,Gym / Fitness Center,Cycle Studio,Yoga Studio,Climbing Gym,Residential Building (Apartment / Condo),Gym Pool,Medical Center,Athletics & Sports,Boxing Gym
16,Murray Hill,Gym,Gym / Fitness Center,Yoga Studio,Pilates Studio,Medical Center,Cycle Studio,Boxing Gym,Corporate Amenity,Athletics & Sports,General College & University
21,Tribeca,Gym,Gym / Fitness Center,Track,Yoga Studio,Pilates Studio,Athletics & Sports,Gym Pool,Gymnastics Gym,Office,Cycle Studio
24,West Village,Gym,Gym / Fitness Center,Cycle Studio,Yoga Studio,Pool,Martial Arts Dojo,Track,Pilates Studio,Boxing Gym,Corporate Amenity
25,Manhattan Valley,Gym / Fitness Center,Gym,Yoga Studio,Martial Arts Dojo,Track,Corporate Amenity,Athletics & Sports,General College & University,Doctor's Office,Dance Studio
27,Gramercy,Gym,Gym / Fitness Center,Yoga Studio,Pilates Studio,Martial Arts Dojo,Doctor's Office,Gym Pool,Corporate Amenity,Athletics & Sports,General College & University
29,Financial District,Gym,Gym / Fitness Center,Boxing Gym,Doctor's Office,Medical Center,Cycle Studio,Pilates Studio,Yoga Studio,Weight Loss Center,College Gym


## Cluster 4

In [33]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Gym,Gym / Fitness Center,Pilates Studio,Community Center,General College & University,Doctor's Office,Dance Studio,Cycle Studio,Corporate Amenity,Yoga Studio
11,Roosevelt Island,Gym,Gym / Fitness Center,Yoga Studio,Community Center,General College & University,Doctor's Office,Dance Studio,Cycle Studio,Corporate Amenity,College Gym
14,Clinton,Gym,Gym / Fitness Center,Yoga Studio,Tennis Court,Building,Residential Building (Apartment / Condo),Basketball Court,Corporate Amenity,Athletics & Sports,General College & University
15,Midtown,Gym,Gym / Fitness Center,Pilates Studio,Yoga Studio,Cycle Studio,Hotel,Boxing Gym,Athletics & Sports,Medical Center,Bridge
28,Battery Park City,Gym,Gym / Fitness Center,Yoga Studio,Office,Boxing Gym,Corporate Amenity,Cycle Studio,Gym Pool,Gymnastics Gym,Recreation Center


## Cluster 5

In [34]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Gym / Fitness Center,Gym,Yoga Studio,Pilates Studio,Cycle Studio,Doctor's Office,Women's Store,Martial Arts Dojo,Medical Center,Track
10,Lenox Hill,Gym / Fitness Center,Gym,Pilates Studio,Yoga Studio,Cycle Studio,Non-Profit,Doctor's Office,Spa,Dance Studio,Recreation Center
17,Chelsea,Gym / Fitness Center,Yoga Studio,Gym,Martial Arts Dojo,Cycle Studio,Recreation Center,Climbing Gym,Pilates Studio,Gymnastics Gym,Spa
26,Morningside Heights,Gym / Fitness Center,Gym,Yoga Studio,Track,Gym Pool,Cycle Studio,Basketball Court,Boxing Gym,Bridge,Building
32,Civic Center,Gym / Fitness Center,Gym,Yoga Studio,Boxing Gym,Martial Arts Dojo,Athletics & Sports,Pilates Studio,Gym Pool,Women's Store,Medical Center
33,Midtown South,Gym / Fitness Center,Gym,Yoga Studio,Martial Arts Dojo,Pilates Studio,Cycle Studio,Building,Boxing Gym,Physical Therapist,Corporate Amenity
38,Flatiron,Gym / Fitness Center,Gym,Cycle Studio,Yoga Studio,Pilates Studio,Boxing Gym,Bridge,Building,Climbing Gym,Club House


# Clustering based on any kind of venue categories

In [42]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering_all = manhattan_all_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans_all = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering_all)

# check cluster labels generated for each row in the dataframe
kmeans_all.labels_[0:10] 

array([1, 2, 2, 2, 2, 1, 2, 2, 2, 4])

In [43]:
# add clustering labels
neighborhoods_venues_sorted_all.insert(0, 'Cluster Labels', kmeans_all.labels_)

manhattan_merged_all = manhattan_data
manhattan_merged_all = manhattan_merged_all.join(neighborhoods_venues_sorted_all.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged_all.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,2,Deli / Bodega,Salon / Barbershop,Residential Building (Apartment / Condo),High School,Laundry Service,Church,Furniture / Home Store,Nail Salon,Office,Pharmacy
1,Manhattan,Chinatown,40.715618,-73.994279,2,Chinese Restaurant,Miscellaneous Shop,Bus Station,Bakery,Bridge,Noodle House,Park,Arts & Crafts Store,Bus Line,Ice Cream Shop
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Residential Building (Apartment / Condo),Doctor's Office,Building,Bike Shop,Salon / Barbershop,Playground,Miscellaneous Shop,Lounge,Factory,Pharmacy
3,Manhattan,Inwood,40.867684,-73.921210,2,Laundry Service,Deli / Bodega,Salon / Barbershop,Nail Salon,American Restaurant,Bank,Mexican Restaurant,Pharmacy,Bus Line,Doctor's Office
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Residential Building (Apartment / Condo),Salon / Barbershop,Building,Deli / Bodega,Non-Profit,Laundry Service,Church,Bar,Bus Station,Caribbean Restaurant


In [46]:

# create map
map_clusters_all = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged_all['Latitude'], manhattan_merged_all['Longitude'], manhattan_merged_all['Neighborhood'], manhattan_merged_all['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_all)
       
map_clusters_all

## Cluster 1

In [47]:
manhattan_merged_all.loc[manhattan_merged_all['Cluster Labels'] == 0, manhattan_merged_all.columns[[1] + list(range(5, manhattan_merged_all.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Residential Building (Apartment / Condo),Doctor's Office,Building,Bike Shop,Salon / Barbershop,Playground,Miscellaneous Shop,Lounge,Factory,Pharmacy
8,Upper East Side,Doctor's Office,Residential Building (Apartment / Condo),Art Gallery,Building,Medical Center,Dentist's Office,Office,Library,Sculpture Garden,Medical Lab
16,Murray Hill,Residential Building (Apartment / Condo),Doctor's Office,Dentist's Office,Building,Spa,Hotel,Event Space,Office,Embassy / Consulate,Other Great Outdoors
27,Gramercy,Doctor's Office,Building,Residential Building (Apartment / Condo),Medical Center,Dentist's Office,Laundry Service,Salon / Barbershop,Gourmet Shop,School,Coworking Space
36,Tudor City,Doctor's Office,Residential Building (Apartment / Condo),Medical Center,Building,Gym / Fitness Center,Bus Stop,Laundry Service,Veterinarian,Deli / Bodega,Playground


## Cluster 2

In [49]:
manhattan_merged_all.loc[manhattan_merged_all['Cluster Labels'] == 1, manhattan_merged_all.columns[[1] + list(range(5, manhattan_merged_all.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Manhattanville,College Administrative Building,Automotive Shop,Bank,Building,Residential Building (Apartment / Condo),Deli / Bodega,Medical Center,University,Food Truck,Pharmacy
21,Tribeca,Office,Coworking Space,Residential Building (Apartment / Condo),Food Truck,Café,Building,General Entertainment,Salad Place,Bank,Shoe Store
22,Little Italy,Italian Restaurant,Office,Coworking Space,Gift Shop,Bakery,Salon / Barbershop,Pharmacy,Gourmet Shop,Deli / Bodega,Building
26,Morningside Heights,Food Truck,General College & University,College Academic Building,Student Center,College Administrative Building,College Residence Hall,Office,College Library,College Theater,College Quad
28,Battery Park City,Residential Building (Apartment / Condo),Boat or Ferry,Park,Harbor / Marina,Office,Food Truck,Building,Hot Dog Joint,Bar,Dog Run
32,Civic Center,Office,Government Building,Building,Gym,Doctor's Office,Food Truck,Deli / Bodega,Lawyer,Residential Building (Apartment / Condo),Convenience Store
33,Midtown South,Office,Spa,Food Stand,American Restaurant,Residential Building (Apartment / Condo),Department Store,Building,Lounge,Gym,Sandwich Place
34,Sutton Place,Residential Building (Apartment / Condo),Office,Parking,Gym / Fitness Center,Bus Line,Medical Center,Laundry Service,Gym,Furniture / Home Store,Food Truck
35,Turtle Bay,Government Building,Office,Building,Residential Building (Apartment / Condo),Embassy / Consulate,Non-Profit,Monument / Landmark,Tunnel,Doctor's Office,Meeting Room


## Cluster 3

In [51]:
manhattan_merged_all.loc[manhattan_merged_all['Cluster Labels'] == 2, manhattan_merged_all.columns[[1] + list(range(5, manhattan_merged_all.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Deli / Bodega,Salon / Barbershop,Residential Building (Apartment / Condo),High School,Laundry Service,Church,Furniture / Home Store,Nail Salon,Office,Pharmacy
1,Chinatown,Chinese Restaurant,Miscellaneous Shop,Bus Station,Bakery,Bridge,Noodle House,Park,Arts & Crafts Store,Bus Line,Ice Cream Shop
3,Inwood,Laundry Service,Deli / Bodega,Salon / Barbershop,Nail Salon,American Restaurant,Bank,Mexican Restaurant,Pharmacy,Bus Line,Doctor's Office
4,Hamilton Heights,Residential Building (Apartment / Condo),Salon / Barbershop,Building,Deli / Bodega,Non-Profit,Laundry Service,Church,Bar,Bus Station,Caribbean Restaurant
6,Central Harlem,Salon / Barbershop,Residential Building (Apartment / Condo),Mountain,Gym / Fitness Center,Bus Line,Bus Stop,Non-Profit,Church,Cocktail Bar,Building
7,East Harlem,Residential Building (Apartment / Condo),Pizza Place,Pharmacy,Hardware Store,Clothing Store,Lawyer,Bus Stop,Salon / Barbershop,Church,Mobile Phone Shop
9,Yorkville,Residential Building (Apartment / Condo),Laundry Service,Building,Pharmacy,Deli / Bodega,Spa,Bus Stop,Salon / Barbershop,Bar,Pizza Place
10,Lenox Hill,Building,Residential Building (Apartment / Condo),Salon / Barbershop,Nail Salon,Deli / Bodega,Art Gallery,Bus Line,Rental Car Location,Parking,Doctor's Office
11,Roosevelt Island,Residential Building (Apartment / Condo),Church,Building,Laundry Service,Rental Car Location,Art Gallery,Playground,Restaurant,Doctor's Office,Deli / Bodega
12,Upper West Side,Residential Building (Apartment / Condo),Cosmetics Shop,Laundry Service,Nail Salon,Pizza Place,Jewelry Store,Bus Line,Bar,Furniture / Home Store,Doctor's Office


## Cluster 4

In [53]:
manhattan_merged_all.loc[manhattan_merged_all['Cluster Labels'] == 3, manhattan_merged_all.columns[[1] + list(range(5, manhattan_merged_all.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,Hudson Yards,Bus Stop,Furniture / Home Store,Food Truck,Transportation Service,Meeting Room,Moving Target,Bike Rental / Bike Share,Tattoo Parlor,Bus Station,Automotive Shop


## cluster 5

In [55]:
manhattan_merged_all.loc[manhattan_merged_all['Cluster Labels'] == 4, manhattan_merged_all.columns[[1] + list(range(5, manhattan_merged_all.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Midtown,Office,General College & University,Coworking Space,Building,College Administrative Building,American Restaurant,Shoe Store,University,College Arts Building,Tailor Shop
23,Soho,Office,Boutique,Clothing Store,Women's Store,Design Studio,Art Gallery,Tech Startup,Event Space,Music Venue,Jewelry Store
29,Financial District,Office,Building,Tech Startup,Historic Site,Gym / Fitness Center,Non-Profit,Doctor's Office,Financial or Legal Service,Juice Bar,Food Truck
38,Flatiron,Office,Tech Startup,Building,Women's Store,Boutique,Clothing Store,Sporting Goods Shop,Gym / Fitness Center,Furniture / Home Store,Cosmetics Shop


# Analysis

Now let us analyze some results. If we consider neighborhood 'Chelsea'. It appears in cluster5 when taking into account venue category = gym/fitness center. This cluster is characterized by plenty gym/fitness centers. 

However, if we search for 'Chelsea' in the second stage of clustering when we take into account all categories, it appears in cluster 3. A closer look to this cluster reveals the absence of vegetarian restaurants. So, we can conclude that Chelsea is a good location to open a new restaurant.